# Computing distribution for 1-dimensional moduli space

In [ ]:
import plotly.graph_objects as go

from singdistance import *

logger.setLevel(logging.DEBUG)

In [ ]:
phi1 = .45 * np.pi / 3
alpha1, alpha2 = .4 * np.pi / 3, .5 * np.pi / 3

phi2 = alpha1 + alpha2 - phi1

limits = max(0, phi1 - alpha2), min(alpha1, phi1)

def area(_beta):
    _beta_ = phi1 - _beta
    _beta2 = alpha1 - _beta
    _beta2_ = phi2 - _beta2
    a1 = 1 / (1 / (np.tan(_beta / 2 + 1e-6)) + 1 / np.tan(_beta2 / 2 + 1e-6))
    a2 = 1 / (1 / np.tan(_beta_ / 2 + 1e-6) + 1 / np.tan(_beta2_ / 2 + 1e-6))
    return a1 + a2

if limits[0] < limits[1]:
    area0, area1 = area(limits[0]), area(limits[1])
else:
    area0, area1 = 0, 0

m = np.exp(-1j * alpha1 / 2) / np.sin(alpha1 / 2) + \
    np.exp(1j * (alpha2 / 2 - phi1)) / np.sin(alpha2 / 2)
m = np.abs(m) / 2
c = - 1 / np.tan(alpha1 / 2) / 2 - 1 / np.tan(alpha2 / 2) / 2
q = 1 / np.tan(phi1 / 2) + 1 / np.tan(phi2 / 2)

def f_an(a):
    # https://tinyurl.com/ansolution

    w1 = ((1 - c * q)**2 - m**2 * q**2) ** .5
    w2 = (a - c) * (1 - c * q) - m**2 * q
    w3 = (m**2 - (a - c)**2) ** .5

    res = np.arctan(w2/w3/w1) / w1 / q / a**2
    res += np.arctan((c - a) / w3) / q / a**2
    return res

k0 = -f_an(area0) / 2 / area0**-2
k1 = -f_an(area1) / 2 / area1**-2

def f_res(a):
    if limits[0] > limits[1]:
        return 0
    res = 0
    if m + c > a > area0:
        res += f_an(a) / 2
    if m + c > a > area1:
        res += f_an(a) / 2
    if q < 0 or a < 1 / q:
        if a > area0:
            res += k0 * a**-2
        if a > area1:
            res += k1 * a**-2
    return res

In [ ]:
m1 = ModuliSpace(phi1, (alpha1, alpha2))
m1.ode_steps = 10000
m1.calc_area_distribution()

x = m1.a_distribution.steps[:-1]
y = m1.a_distribution.values

fig = go.Figure()
fig.add_trace(go.Scatter(x=x, y=y))
fig.add_trace(go.Scatter(x=x, y=[f_res(x) for x in x]))
fig['layout'].update(margin=dict(l=0, r=0, b=0, t=0))
fig.show()